# Plant species recommender system - iteration 2
This is the second iteration of the recommender system. Plant species that contribute positively to biodiversity are recommended based on the plant species the user already has in their garden, the characteristics of their garden, and criteria chosen by the user.
In this NoteBook, a user profile, garden profile, and user criteria are used. When necessary, example inputs are printed. These can be used to run the system. However, other inputs may be used if desired, but they must match with plant species in the database (recommender_system.csv). Otherwise the system cannot work.

# 1. Recommend plant species similar to those present in the user's garden

### 1.1 Import libraries

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import Image
import numpy as np
import pandas as pd

### 1.2 Import CSV file with plant data
This CSV was generated earlier as a result of web scraping and data cleaning in the .py files

In [31]:
# Create a dataframe from the CSV file with plant data
df_plants = pd.read_csv("recommender_system.csv", encoding="UTF-8")

# 2. Collect data from user
## 2.1 Create user profile
The user profile is filled in using the information given by the user when using the application. When a user adds a plant species in their garden, it is added to this list. However it is only possible to add plants known by the application (i.e. present in the CSV file).

In [32]:
list_user = [ ] 
number_plants = int(input("How many plants do you already have in your garden? "+"\033[1m"+"(for example 3)"+"\033[0m"))
  
for i in range(0, number_plants): 
    print("Type the name of a plant that is in your garden. Note that the recommender system only works with plants that are in its database.")
    print("\033[1m"+"Examples are: "+"\033[0m")
    print("\033[1m"+"Plant 1: Pontische Rododendron Rhododendron ponticum Roseum"+"\033[0m")
    print("\033[1m"+"Plant 2: Blauwe regen Wisteria sinensis Boskoop"+"\033[0m")
    print("\033[1m"+"Plant 3: Franse lavendel Lavandula stoechas subsp pedunculata"+"\033[0m")
    input_user = input("Enter the name of one of your plants: ")
    list_user.append(input_user) 
      
print("The user has the following plants already in his garden: ", list_user)

How many plants do you already have in your garden? (for example 3)3
Type the name of a plant that is in your garden. Note that the recommender system only works with plants that are in its database.
Examples are: 
Plant 1: Pontische Rododendron Rhododendron ponticum Roseum
Plant 2: Blauwe regen Wisteria sinensis Boskoop
Plant 3: Franse lavendel Lavandula stoechas subsp pedunculata
Enter the name of one of your plants: Pontische Rododendron Rhododendron ponticum Roseum
Type the name of a plant that is in your garden. Note that the recommender system only works with plants that are in its database.
Examples are: 
Plant 1: Pontische Rododendron Rhododendron ponticum Roseum
Plant 2: Blauwe regen Wisteria sinensis Boskoop
Plant 3: Franse lavendel Lavandula stoechas subsp pedunculata
Enter the name of one of your plants: Blauwe regen Wisteria sinensis Boskoop
Type the name of a plant that is in your garden. Note that the recommender system only works with plants that are in its database.
Ex

## 2.2 Create garden profile
The garden profile contains factors to consider when planting a plant species. It is important to plant plant species that match the growing conditions offered by the garden.

In [33]:
# Creating dictionary with the garden criteria of the user
dict_garden = {
            "Sun exposure": "",
            "Soil moisture": ""
}
print("The profile of your garden includes its exposure to the sun and the soil's moisture.")
print("\033[1m"+"The sun exposure can be one of the following: Zon, Lichte schaduw, Halfschaduw, Schaduw"+"\033[0m")
print("\033[1m"+"Example: Halfschaduw"+"\033[0m")
dict_garden["Sun exposure"] = input("Enter the sun exposure of your garden: ")

print("\033[1m"+"The soil moisture can be one of the following: Droog, Vochthoudend, Vochtig"+"\033[0m")
print("\033[1m"+"Example: Vochthoudend"+"\033[0m")
dict_garden["Soil moisture"] = input("Enter the soil moisture of your garden: ")


# Convert dictionary to two lists
Sun_exposure = list(dict_garden.items())[0][1]
print('Sun exposure of the garden: ', Sun_exposure)
Soil_moisture = list(dict_garden.items())[1][1]
print('Soil moisture of the garden: ', Soil_moisture)

The profile of your garden includes its exposure to the sun and the soil's moisture.
The sun exposure can be one of the following: Zon, Lichte schaduw, Halfschaduw, Schaduw
Example: Halfschaduw
Enter the sun exposure of your garden: Halfschaduw
The soil moisture can be one of the following: Droog, Vochthoudend, Vochtig
Example: Vochthoudend
Enter the soil moisture of your garden: Vochthoudend
Sun exposure of the garden:  Halfschaduw
Soil moisture of the garden:  Vochthoudend


## 2.3 Collect user criteria
When the user searches for a new plant species, they has the possibility to enter plant criteria that are important to them. Only the plants corresponding to their criteria will be recommended. The user's criteria can cover all the plant properties. 

In [35]:
dict_user = {
            "Type": "",
            "Birds": "",
            "Bees": "",
            "Butterflies": "",
            "Hedgehogs": "",
            "Squirrels": "",
            "Flowering period": "",
            "Leaf color": "",
            "Flower color": "",
            "Evergreen": "",
            "Plant growth": "",
            "Maintenance": ""
}
print("There are 12 criteria from which you can determine a requirement.")
number_plants = int(input("How many criteria do you want the recommender system to take into account?"+"\033[1m"+"(for example 2)"+"\033[0m")) 

for i in range(number_plants):
    print("The criteria are: Type, Birds, Bees, Butterflies, Hedgehogs, Squirrels, Flowering period, Leaf color, Flower color, Evergreen, Plant growth, Maintenance.")
    print("\033[1m"+"Examples are: "+"\033[0m")
    print("\033[1m"+"Type: Borderplanten"+"\033[0m")
    print("\033[1m"+"Bees: Bijen"+"\033[0m")
    dict_key = input("Enter which of the criteria you wish to include "+"\033[1m"+"(for example Type)"+"\033[0m"+": ")
    dict_value = input("Enter the value of the criteria "+"\033[1m"+"(for example Borderplanten)"+"\033[0m"+": ")
    dict_user[dict_key.title()] = dict_value

print("The user chose the following criteria: ",dict_user)


There are 12 criteria from which you can determine a requirement.
How many criteria do you want the recommender system to take into account?(for example 2)2
The criteria are: Type, Birds, Bees, Butterflies, Hedgehogs, Squirrels, Flowering period, Leaf color, Flower color, Evergreen, Plant growth, Maintenance.
Examples are: 
Type: Borderplanten
Bees: Bijen
Enter which of the criteria you wish to include (for example Type): Type
Enter the value of the criteria (for example Borderplanten): Borderplanten
The criteria are: Type, Birds, Bees, Butterflies, Hedgehogs, Squirrels, Flowering period, Leaf color, Flower color, Evergreen, Plant growth, Maintenance.
Examples are: 
Type: Borderplanten
Bees: Bijen
Enter which of the criteria you wish to include (for example Type): Bees
Enter the value of the criteria (for example Borderplanten): Bijen
The user chose the following criteria:  {'Type': 'Borderplanten', 'Birds': '', 'Bees': 'Bijen', 'Butterflies': '', 'Hedgehogs': '', 'Squirrels': '', 'Flo

# 3. Plant similarities
## 3.1 Transform text to feature vectors
Transforming the text of all cells into vectors (numerical values) will allow to measure the similarities between the plant species already present in the user's garden with all the other plant species in the application's database (CSV file). To do this, all columns must be merged into one. Some columns like the Name column are not taken into account, because it is not part of the plant properties.

In [36]:
# Create a copy of df_plants to avoid unwanted errors
df_copy = df_plants.copy()
df_copy = df_copy.drop(["Name"], axis=1)
df_copy = df_copy.drop(["Unnamed: 0"], axis=1)
df_copy = df_copy.drop(["Photo"], axis=1)
df_copy = df_copy.drop(["Maintenance description"], axis=1)

# Create a new df with 1 column that contains the values of all columns
df_recom = pd.DataFrame()
df_recom["Recommendation"]= df_copy[df_copy.columns[1:]].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
df_recom

,Recommendation
0,Borderplanten GeenVogels Bijen Vlinders GeenEg...
1,Borderplanten Vogels GeenBijen GeenVlinders Ge...
2,Borderplanten GeenVogels GeenBijen Vlinders Ge...
3,Borderplanten GeenVogels Bijen Vlinders GeenEg...
4,Borderplanten GeenVogels Bijen Vlinders GeenEg...
...,...
926,Heesters en Struiken Vogels GeenBijen GeenVlin...
927,Heesters en Struiken Vogels Bijen Vlinders Gee...
928,Borderplanten GeenVogels Bijen Vlinders GeenEg...
929,Heesters en Struiken Vogels Bijen Vlinders Gee...


In [37]:
# Add a column ID with an ID for each row
df_recom["ID"]=[i for i in range(0,df_recom.shape[0])]

# Merge the two df df_plants and df_recom to retrieve the plant species names
df_recom = pd.merge(df_plants, df_recom, on="ID")
df_recom = df_recom.drop(["Unnamed: 0"], axis=1)
df_recom

,ID,Name,Type,Birds,Bees,Butterflies,Hedgehogs,Squirrels,Flowering period,Leaf color,Flower color,Evergreen,Plant growth,Sun exposure,Soil moisture,Maintenance,Maintenance description,Photo,Recommendation
0,0,Aar ereprijs Veronica spicata,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Juli Augustus,BladGroen,BloemWit,NietWintergroen,50.0,Zon,Droog vochthoudend,Laag,De plant sterft elke winter af en loopt in het...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:...",Borderplanten GeenVogels Bijen Vlinders GeenEg...
1,1,Adderwortel Persicaria bistorta,Borderplanten,Vogels,GeenBijen,GeenVlinders,GeenEgels,GeenEekhoorns,Mei Juni Juli,BladGroen,BloemRoze,NietWintergroen,70.0,Lichteschaduw Zon,Vochthoudend vochtig,Laag,De plant sterft elke winter af en loopt in het...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:...",Borderplanten Vogels GeenBijen GeenVlinders Ge...
2,2,Afrikaanse lelie Agapanthus,Borderplanten,GeenVogels,GeenBijen,Vlinders,GeenEgels,GeenEekhoorns,Juli Augustus,BladGroen,BloemBlauw,NietWintergroen,100.0,Zon,Droog vochthoudend,Laag,De plant sterft elke winter af en loopt in het...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:...",Borderplanten GeenVogels GeenBijen Vlinders Ge...
3,3,Akelei Aquilegia viridiflora Chocolate Soldier,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Mei Juni,BladGroen,BloemPaars BloemBruin,NietWintergroen,60.0,Schaduw,Vochthoudend,Geen,De plant heeft geen bijzondere zorg nodig. Zaa...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:...",Borderplanten GeenVogels Bijen Vlinders GeenEg...
4,4,Alpenaster Aster alpinus Happy End,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Mei Juni,BladGroen,BloemRoze,Wintergroen,20.0,Zon,Vochthoudend vochtig,Laag,De plant sterft elke winter af en loopt in het...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:...",Borderplanten GeenVogels Bijen Vlinders GeenEg...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926,926,Zuurbes Berberis x lologensis Apricot Queen,Heesters en Struiken,Vogels,GeenBijen,GeenVlinders,GeenEgels,GeenEekhoorns,Mei Juni,BladGroen,BloemGeel BloemOranje,Wintergroen,125.0,Lichteschaduw Zon,Vochthoudend vochtig,Geen,De plant heeft geen bijzondere zorg nodig.,"https://cdn.zilvercms.nl/330x221,fit,q80/http:...",Heesters en Struiken Vogels GeenBijen GeenVlin...
927,927,Zwarte aalbes Ribes nigrum Titania,Heesters en Struiken,Vogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,April Mei Juli,BladGroen,BloemWit,NietWintergroen,200.0,"Halfschaduw, Zon",Vochthoudend,Hoog,Na het planten worden de takken voor 1/3 inges...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:...",Heesters en Struiken Vogels Bijen Vlinders Gee...
928,928,Zwarte ereprijs Veronicastrum virginicum Diana,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Juli Augustus,BladGroen,BloemWit,NietWintergroen,150.0,Zon,Droog vochthoudend,Laag,De plant sterft elke winter af en loopt in het...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:...",Borderplanten GeenVogels Bijen Vlinders GeenEg...
929,929,Zwarte toorts Verbascum nigrum,Heesters en Struiken,Vogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,September Augustus Juli Juni,BladGroen,BloemGeel,NietWintergroen,100.0,Zon,Droog,Laag,"De plant is groenblijvend, snoei is niet nodig...","https://cdn.zilvercms.nl/330x221,fit,q80/http:...",Heesters en Struiken Vogels Bijen Vlinders Gee...


In [38]:
# Text to vector transformation using Term Frequency - Inverse Document Frequency vectorization
vecs=TfidfVectorizer().fit_transform(df_recom["Recommendation"].apply(lambda x: np.str_(x)))
vecs

<931x117 sparse matrix of type '<class 'numpy.float64'>'
	with 15870 stored elements in Compressed Sparse Row format>

## 3.2 Calculate the cosine similarity
The cosine similarity can be computed from the feature vectors to know the degree of similarity between two feature vectors.

In [39]:
sim=cosine_similarity(vecs) 

## 3.3 Get the list of plants similar to those in the user's garden

In [40]:
def recommend(user_plants):
    # Get the user's plant species ID
    id_plants=df_recom[df_recom["Name"]==user_plants]["ID"].values[0]
    
    # Calculates the similarity between user's plant species and all other plant species in the application's database
    scores=list(enumerate(sim[id_plants]))
    
    # Sort plant species in ascending order
    sorted_scores=sorted(scores,key=lambda x:x[1],reverse=True)
    
    # Get the name of similar plant species from their ID
    plants=[df_recom[plants[0]==df_recom["ID"]]["Name"].values[0] for plants in sorted_scores]
    
    return plants

In [41]:
# Run the function for each plant species that the user has in his garden
for plant in list_user:
    rec_plants=recommend(plant)
# Remove the plant species that the user already has in his garden from the list of recommended plant species    
    for plant in list_user:
        if plant in list_user:
            rec_plants.remove(plant)

In [42]:
print('These are the 5 plant species that are most similar to the plant species the user already has in his garden: ',
      rec_plants[:5])

These are the 5 plant species that are most similar to the plant species the user already has in his garden:  ['Lavendel Lavandula', 'Lavendel Lavandula wit', 'Steenraket Erysimum linifolium Bowles Mauve', 'Fijnstraal Erigeron Wuppertal', 'Salie Salvia x sylvestris Mainacht']


## 3.4 Create a ranking indicator
The similarity of the plant species with those in the user's garden must not be lost. This is supported by a column "Ranking".

In [43]:
df_ranking = pd.DataFrame(rec_plants)
# Creation of a "Ranking" column indicating the ranking of the plant species
df_ranking['Ranking'] = df_ranking.reset_index().index
df_ranking.rename(columns = {0:"Name"}, inplace = True)
df_ranking

,Name,Ranking
0,Lavendel Lavandula,0
1,Lavendel Lavandula wit,1
2,Steenraket Erysimum linifolium Bowles Mauve,2
3,Fijnstraal Erigeron Wuppertal,3
4,Salie Salvia x sylvestris Mainacht,4
...,...,...
923,Trosvlier Sambucus racemosa,923
924,Trosvlier Sambucus racemosa Plumosa Aurea,924
925,Trosvlier Sambucus racemosa Sutherland Gold,925
926,Zuilbeuk Fagus sylvatica Dawyck,926


## 3.5 Create a complete dataframe


In [44]:
# Creation of a dataframe including the classification of plant species and all their properties
df_rec = pd.merge(df_ranking, df_plants, on="Name", how="inner")
df_rec = df_rec.drop(["Unnamed: 0"], axis=1)
df_rec

,Name,Ranking,ID,Type,Birds,Bees,Butterflies,Hedgehogs,Squirrels,Flowering period,Leaf color,Flower color,Evergreen,Plant growth,Sun exposure,Soil moisture,Maintenance,Maintenance description,Photo
0,Lavendel Lavandula,0,466,Borderplanten,Vogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Juni Juli,BladGrijs,BloemPaars BloemBlauw,Wintergroen,90.0,Zon,Droog,Gemiddeld,De plant hoeft niet echt gesnoeid te worden. O...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
1,Lavendel Lavandula wit,1,467,Borderplanten,Vogels,GeenBijen,GeenVlinders,GeenEgels,GeenEekhoorns,Juni Juli,BladGrijs,BloemWit,Wintergroen,60.0,Zon,Droog,Gemiddeld,De plant hoeft niet echt gesnoeid te worden. O...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
2,Steenraket Erysimum linifolium Bowles Mauve,2,735,Borderplanten,Vogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,September Augustus Juli Juni Mei,BladGroen BladGrijs,BloemPaars,Wintergroen,60.0,Zon,Droog vochthoudend,Geen,De plant heeft geen bijzondere zorg nodig.,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
3,Fijnstraal Erigeron Wuppertal,3,148,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Augustus Juli Juni,BladGroen,BloemPaars BloemBlauw,NietWintergroen,60.0,Zon,Droog vochthoudend,Laag,De plant sterft elke winter af en loopt in het...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
4,Salie Salvia x sylvestris Mainacht,4,660,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Juli Juni Mei,BladGroen,BloemPaars BloemBlauw,NietWintergroen,60.0,Zon,Droog vochthoudend,Laag,De plant sterft elke winter af en loopt in het...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,Trosvlier Sambucus racemosa,923,784,Heesters en Struiken,Vogels,GeenBijen,GeenVlinders,GeenEgels,GeenEekhoorns,April Mei,BladGroen,BloemGeel BloemGroen,NietWintergroen,300.0,Lichteschaduw Halfschaduw,Vochthoudend vochtig,Geen,De plant heeft geen bijzondere zorg nodig.,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
924,Trosvlier Sambucus racemosa Plumosa Aurea,924,785,Heesters en Struiken,Vogels,GeenBijen,GeenVlinders,GeenEgels,GeenEekhoorns,April Mei,BladGeel,BloemGeel BloemGroen,NietWintergroen,250.0,Lichteschaduw Halfschaduw,Vochthoudend vochtig,Geen,De plant heeft geen bijzondere zorg nodig.,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
925,Trosvlier Sambucus racemosa Sutherland Gold,925,786,Heesters en Struiken,Vogels,GeenBijen,GeenVlinders,GeenEgels,GeenEekhoorns,Mei April,BladGroen BladGeel,BloemGeel BloemGroen,NietWintergroen,250.0,Lichteschaduw Halfschaduw,Vochthoudend vochtig,Geen,De plant heeft geen bijzondere zorg nodig.,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
926,Zuilbeuk Fagus sylvatica Dawyck,926,923,Bomen,GeenVogels,GeenBijen,GeenVlinders,GeenEgels,Eekhoorns,Mei,BladGroen BladBruin,NaN,NietWintergroen,4000.0,NaN,NaN,NaN,NaN,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."


# 4. Criteria of the user's garden
The criteria of the user's garden must be taken into account in order to have plants that will be able to grow in optimal conditions.

# 4. Filter recommended plants based on garden profile
Only plants that can grow in the user's garden are kept.

In [45]:
# Keep plant species that met garden criteria
# Sun exposure
df_rec = df_rec[df_rec['Sun exposure'].str.contains(Sun_exposure, case=False, na=False)]
# Soil moisture
df_rec = df_rec[df_rec['Soil moisture'].str.contains(Soil_moisture, case=False, na=False)]

df_rec

,Name,Ranking,ID,Type,Birds,Bees,Butterflies,Hedgehogs,Squirrels,Flowering period,Leaf color,Flower color,Evergreen,Plant growth,Sun exposure,Soil moisture,Maintenance,Maintenance description,Photo
12,Kluwenklokje Campanula glomerata Speciosa,12,393,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Augustus Juli Juni,BladGroen,BloemPaars BloemBlauw,NietWintergroen,60.0,Halfschaduw,Vochthoudend,Laag,De plant sterft elke winter af en loopt in het...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
18,Duifkruid Scabiosa columbaria,18,82,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,September Augustus Juli Juni,BladGroen BladGrijs,BloemBlauw BloemPaars,Wintergroen,40.0,Halfschaduw Zon,Droog vochthoudend,Laag,De plant sterft elke winter af en loopt in het...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
24,Sleutelbloem Primula secundiflora,24,695,Borderplanten,Vogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Mei Juni,BladGroen,BloemPaars,NietWintergroen,60.0,Halfschaduw,Vochthoudend,NaN,De plant heeft geen bijzondere zorg nodig.,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
31,Bergaster Aster amellus,31,15,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Oktober September Augustus,BladGroen,BloemPaars BloemBlauw,NietWintergroen,60.0,Halfschaduw Zon,Droog vochthoudend,Laag,De plant sterft elke winter af en loopt in het...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
32,Sleutelbloem Primula pulverulenta,32,693,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Mei Juni,BladGroen,BloemRoze BloemPaars,Wintergroen,60.0,Halfschaduw,Vochthoudend,Geen,De plant heeft geen bijzondere zorg nodig.,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
918,Sierappel boom Malus toringo,918,674,Fruitstruiken en moestuinplanten,Vogels,GeenBijen,GeenVlinders,GeenEgels,GeenEekhoorns,Mei,BladGroen,BloemWit BloemRoze,NietWintergroen,700.0,"Halfschaduw, zon",Vochthoudend vochtig,Hoog,Pitvruchten kunt u snoeien zolang er geen blad...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
922,rode bruine beukenhaag fagus sylvatica/,922,49,Haagplanten,Vogels,GeenBijen,GeenVlinders,Egels,GeenEekhoorns,NaN,BladBruin BladRood,NaN,NietWintergroen,2000.0,"Halfschaduw, zon",Vochthoudend vochtig,Hoog,Een beukenhaag wordt meestal 2 maal per jaar g...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
923,Trosvlier Sambucus racemosa,923,784,Heesters en Struiken,Vogels,GeenBijen,GeenVlinders,GeenEgels,GeenEekhoorns,April Mei,BladGroen,BloemGeel BloemGroen,NietWintergroen,300.0,Lichteschaduw Halfschaduw,Vochthoudend vochtig,Geen,De plant heeft geen bijzondere zorg nodig.,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
924,Trosvlier Sambucus racemosa Plumosa Aurea,924,785,Heesters en Struiken,Vogels,GeenBijen,GeenVlinders,GeenEgels,GeenEekhoorns,April Mei,BladGeel,BloemGeel BloemGroen,NietWintergroen,250.0,Lichteschaduw Halfschaduw,Vochthoudend vochtig,Geen,De plant heeft geen bijzondere zorg nodig.,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."


# 5. Filter recommended plants based on user criteria
Only plants that meet the user's criteria are kept.

In [46]:
# Print the user's criteria 
print('These are the criteria that the user wants to be met:')
Type = list(dict_user.items())[0][1]
print('User criteria Type: ', Type)

Birds = list(dict_user.items())[1][1]
print('User criteria Birds: ', Birds)

Bees = list(dict_user.items())[2][1]
print('User criteria Bees: ', Bees)

Butterflies = list(dict_user.items())[3][1]
print('User criteria Butterflies: ', Butterflies)

Hedgehogs = list(dict_user.items())[4][1]
print('User criteria Hedgehogs: ', Hedgehogs)

Squirrels = list(dict_user.items())[5][1]
print('User criteria Squirrels: ', Squirrels)

Flowering_period = list(dict_user.items())[6][1]
print('User criteria Flowering period: ', Flowering_period)

Leaf_color = list(dict_user.items())[7][1]
print('User criteria Leaf color: ', Leaf_color)

Flower_color = list(dict_user.items())[8][1]
print('User criteria Flower color: ', Flower_color)

Evergreen = list(dict_user.items())[9][1]
print('User criteria Evergreen: ', Evergreen)

Plant_growth = list(dict_user.items())[10][1]
print('User criteria Plant growth: ', Plant_growth)

Maintenance = list(dict_user.items())[11][1]
print('User criteria Maintenance: ', Maintenance)


These are the criteria that the user wants to be met:
User criteria Type:  Borderplanten
User criteria Birds:  
User criteria Bees:  Bijen
User criteria Butterflies:  
User criteria Hedgehogs:  
User criteria Squirrels:  
User criteria Flowering period:  
User criteria Leaf color:  
User criteria Flower color:  
User criteria Evergreen:  
User criteria Plant growth:  
User criteria Maintenance:  


In [47]:
# Keep plant species that met user criteria
# If the user has not specified a criterion, pass.

if dict_user.get('Type') == '':
    pass
else:
    df_rec = df_rec.loc[df_rec['Type'] == Type]

if dict_user.get('Birds') == '':
    pass
else:
    df_rec = df_rec.loc[df_rec['Birds'] == Birds]
    
if dict_user.get('Bees') == '':
    pass
else:
    df_rec = df_rec.loc[df_rec['Bees'] == Bees]
    
if dict_user.get('Butterflies') == '':
    pass
else:
    df_rec = df_rec.loc[df_rec['Butterflies'] == Butterflies]
    
if dict_user.get('Hedgehogs') == '':
    pass
else:
    df_rec = df_rec.loc[df_rec['Hedgehogs'] == Hedgehogs]

if dict_user.get('Squirrels') == '':
    pass
else:
    df_rec = df_rec.loc[df_rec['Squirrels'] == Squirrels]

if dict_user.get('Flowering period') == '':
    pass
else:
    df_rec = df_rec[df_rec['Flowering period'].str.contains(Flowering_period, case=False, na=False)]

if dict_user.get('Leaf color') == '':
    pass
else:
    df_rec = df_rec[df_rec['Leaf color'].str.contains(Leaf, case=False, na=False)]
    
if dict_user.get('Flower color') == '':
    pass
else:
    df_rec = df_rec[df_rec['Flower color'].str.contains(Flower_color, case=False, na=False)]
    
if dict_user.get('Evergreen') == '':
    pass
else:
    df_rec = df_rec.loc[df_rec['Evergreen'] == Evergreen]

if dict_user.get('Plant growth') == '':
    pass
else:
    df_rec = df_rec.loc[df_rec['Plant growth'] == Plant_growth]

if dict_user.get('Maintenance') == '':
    pass
else:
    df_rec = df_rec.loc[df_rec['Maintenance'] == Maintenance]

df_rec

,Name,Ranking,ID,Type,Birds,Bees,Butterflies,Hedgehogs,Squirrels,Flowering period,Leaf color,Flower color,Evergreen,Plant growth,Sun exposure,Soil moisture,Maintenance,Maintenance description,Photo
12,Kluwenklokje Campanula glomerata Speciosa,12,393,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Augustus Juli Juni,BladGroen,BloemPaars BloemBlauw,NietWintergroen,60.0,Halfschaduw,Vochthoudend,Laag,De plant sterft elke winter af en loopt in het...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
18,Duifkruid Scabiosa columbaria,18,82,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,September Augustus Juli Juni,BladGroen BladGrijs,BloemBlauw BloemPaars,Wintergroen,40.0,Halfschaduw Zon,Droog vochthoudend,Laag,De plant sterft elke winter af en loopt in het...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
24,Sleutelbloem Primula secundiflora,24,695,Borderplanten,Vogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Mei Juni,BladGroen,BloemPaars,NietWintergroen,60.0,Halfschaduw,Vochthoudend,NaN,De plant heeft geen bijzondere zorg nodig.,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
31,Bergaster Aster amellus,31,15,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Oktober September Augustus,BladGroen,BloemPaars BloemBlauw,NietWintergroen,60.0,Halfschaduw Zon,Droog vochthoudend,Laag,De plant sterft elke winter af en loopt in het...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
32,Sleutelbloem Primula pulverulenta,32,693,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Mei Juni,BladGroen,BloemRoze BloemPaars,Wintergroen,60.0,Halfschaduw,Vochthoudend,Geen,De plant heeft geen bijzondere zorg nodig.,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,Dovenetel Lamium orvala Album,733,76,Borderplanten,Vogels,Bijen,GeenVlinders,GeenEgels,GeenEekhoorns,Mei Juni,BladGroen,BloemWit,NietWintergroen,40.0,Halfschaduw Schaduw,Vochthoudend,Laag,"Na de bloei, maar pas als de plant lelijk word...","https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
749,Kruiskruid Ligularia dentata,749,449,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Juli Augustus,BladGroen,BloemGeel BloemOranje,NietWintergroen,100.0,Halfschaduw Schaduw Zon,Vochthoudend vochtig,Geen,De plant heeft geen bijzondere zorg nodig.,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
826,Sleutelbloem Primula Tawny Port,826,686,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,April Mei,BladGroen,BloemRood,NietWintergroen,10.0,Halfschaduw,Vochthoudend,Geen,De plant heeft geen bijzondere zorg nodig.,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
844,Longkruid Pulmonaria officinalis Sissinghurst ...,844,482,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Maart April,BladGroen BladZilvergrijs,BloemWit,NietWintergroen,30.0,"Halfschaduw, Zon",Vochthoudend,Laag,De plant sterft elke winter af en loopt in het...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."


# 6. Results: recommended plant species
The plant species are now ranked based on their similarity with what the user already has in his garden. All the plant species meet the criteria of the user and the criteria of their garden.

In [48]:
# Reset ranking
df_rec['Ranking'] = df_rec.reset_index().index+1
#Reset index
df_rec.index = np.arange(1, len(df_rec)+1)
df_rec


,Name,Ranking,ID,Type,Birds,Bees,Butterflies,Hedgehogs,Squirrels,Flowering period,Leaf color,Flower color,Evergreen,Plant growth,Sun exposure,Soil moisture,Maintenance,Maintenance description,Photo
1,Kluwenklokje Campanula glomerata Speciosa,1,393,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Augustus Juli Juni,BladGroen,BloemPaars BloemBlauw,NietWintergroen,60.0,Halfschaduw,Vochthoudend,Laag,De plant sterft elke winter af en loopt in het...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
2,Duifkruid Scabiosa columbaria,2,82,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,September Augustus Juli Juni,BladGroen BladGrijs,BloemBlauw BloemPaars,Wintergroen,40.0,Halfschaduw Zon,Droog vochthoudend,Laag,De plant sterft elke winter af en loopt in het...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
3,Sleutelbloem Primula secundiflora,3,695,Borderplanten,Vogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Mei Juni,BladGroen,BloemPaars,NietWintergroen,60.0,Halfschaduw,Vochthoudend,NaN,De plant heeft geen bijzondere zorg nodig.,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
4,Bergaster Aster amellus,4,15,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Oktober September Augustus,BladGroen,BloemPaars BloemBlauw,NietWintergroen,60.0,Halfschaduw Zon,Droog vochthoudend,Laag,De plant sterft elke winter af en loopt in het...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
5,Sleutelbloem Primula pulverulenta,5,693,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Mei Juni,BladGroen,BloemRoze BloemPaars,Wintergroen,60.0,Halfschaduw,Vochthoudend,Geen,De plant heeft geen bijzondere zorg nodig.,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Dovenetel Lamium orvala Album,58,76,Borderplanten,Vogels,Bijen,GeenVlinders,GeenEgels,GeenEekhoorns,Mei Juni,BladGroen,BloemWit,NietWintergroen,40.0,Halfschaduw Schaduw,Vochthoudend,Laag,"Na de bloei, maar pas als de plant lelijk word...","https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
59,Kruiskruid Ligularia dentata,59,449,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Juli Augustus,BladGroen,BloemGeel BloemOranje,NietWintergroen,100.0,Halfschaduw Schaduw Zon,Vochthoudend vochtig,Geen,De plant heeft geen bijzondere zorg nodig.,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
60,Sleutelbloem Primula Tawny Port,60,686,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,April Mei,BladGroen,BloemRood,NietWintergroen,10.0,Halfschaduw,Vochthoudend,Geen,De plant heeft geen bijzondere zorg nodig.,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."
61,Longkruid Pulmonaria officinalis Sissinghurst ...,61,482,Borderplanten,GeenVogels,Bijen,Vlinders,GeenEgels,GeenEekhoorns,Maart April,BladGroen BladZilvergrijs,BloemWit,NietWintergroen,30.0,"Halfschaduw, Zon",Vochthoudend,Laag,De plant sterft elke winter af en loopt in het...,"https://cdn.zilvercms.nl/330x221,fit,q80/http:..."


In [49]:
print('These are the top 5 plant species the recommender system recommends to the user: ',
df_rec["Name"].values[:5])

These are the top 5 plant species the recommender system recommends to the user:  ['Kluwenklokje Campanula glomerata Speciosa'
 'Duifkruid Scabiosa columbaria' 'Sleutelbloem Primula secundiflora'
 'Bergaster Aster amellus' 'Sleutelbloem Primula pulverulenta']


In [50]:
print('\033[1m'+'First recommended plant:'+ '\033[0m')

print("The name of the plant is","\033[1m"+ df_rec.iloc[0]["Name"]+ "\033[0m")
print("The type of the plant is", "\033[1m"+df_rec.iloc[0]['Type']+ "\033[0m")
print("The plant attracts", "\033[1m"+df_rec.iloc[0]['Birds']+", "+df_rec.iloc[0]['Bees']+", "+df_rec.iloc[0]['Butterflies']+", "+df_rec.iloc[0]['Hedgehogs']+" and "+df_rec.iloc[0]['Squirrels']+ "\033[0m")

if not pd.isna(df_rec.iloc[0]['Flowering period']):
    print("The flowering period of the plant is", "\033[1m"+df_rec.iloc[0]['Flowering period']+ "\033[0m")
else:
    print("The flowering period of the plant is", "\033[1m"+"unknown"+"\033[0m")


df_rec['Leaf color'] = df_rec['Leaf color'].str.replace('Blad','')
if not pd.isna(df_rec.iloc[0]['Leaf color']):
    print("The leaf color of the plant is", "\033[1m"+df_rec.iloc[0]['Leaf color']+ "\033[0m")
else:
    print("The leaf color of the plant is", "\033[1m"+"unknown"+"\033[0m")

df_rec['Flower color'] = df_rec['Flower color'].str.replace('Bloem','')
if not pd.isna(df_rec.iloc[0]['Flower color']):
    print("The flower color of the plant is", "\033[1m"+df_rec.iloc[0]['Flower color']+ "\033[0m")
else:
    print("The flower color of the plant is", "\033[1m"+"unknown"+"\033[0m")


if not pd.isna(df_rec.iloc[0]['Evergreen']):
    print("In the winter, the plant is", "\033[1m"+df_rec.iloc[0]['Evergreen']+ "\033[0m")
else:
    print("Whether the plant is evergreen or not is", "\033[1m"+"unknown"+"\033[0m")
    
if not pd.isna(df_rec.iloc[0]['Plant growth']):
    print("The maximum growth height is","\033[1m"+ df_rec.iloc[0]['Plant growth'].astype(str),"cm"+ "\033[0m")
else:
    print("The maximum growth height is", "\033[1m"+"unknown"+"\033[0m")

if not pd.isna(df_rec.iloc[0]['Maintenance']):
    print("The required maintenance level is", "\033[1m"+df_rec.iloc[0]['Maintenance']+ "\033[0m")
else:
    print("The required maintenance level is", "\033[1m"+"unknown"+"\033[0m")
    
    
if not pd.isna(df_rec.iloc[0]['Maintenance description']):
    print("To properly maintain the plant, you must do the following:", "\033[1m"+df_rec.iloc[0]['Maintenance description']+ "\033[0m")
else:
    print("What you need to do to maintain your plant is", "\033[1m"+"unknown"+"\033[0m")


print("This is a picture of the plant:")
Image(url = df_rec.iloc[0]['Photo'])

First recommended plant:
The name of the plant is Kluwenklokje Campanula glomerata Speciosa
The type of the plant is Borderplanten
The plant attracts GeenVogels, Bijen, Vlinders, GeenEgels and GeenEekhoorns
The flowering period of the plant is Augustus Juli Juni
The leaf color of the plant is Groen
The flower color of the plant is Paars Blauw
In the winter, the plant is NietWintergroen
The maximum growth height is 60.0 cm
The required maintenance level is Laag
To properly maintain the plant, you must do the following: De plant sterft elke winter af en loopt in het voorjaar weer uit. Net voor het uitlopen kan het oude loof afgeknipt of weggehaald worden.
This is a picture of the plant:


In [51]:
print('\033[1m'+'Second recommended plant:'+ '\033[0m')

print("The name of the plant is","\033[1m"+ df_rec.iloc[1]["Name"]+ "\033[0m")
print("The type of the plant is", "\033[1m"+df_rec.iloc[1]['Type']+ "\033[0m")
print("The plant attracts", "\033[1m"+df_rec.iloc[1]['Birds']+", "+df_rec.iloc[1]['Bees']+", "+df_rec.iloc[1]['Butterflies']+", "+df_rec.iloc[1]['Hedgehogs']+" and "+df_rec.iloc[1]['Squirrels']+ "\033[0m")

if not pd.isna(df_rec.iloc[1]['Flowering period']):
    print("The flowering period of the plant is", "\033[1m"+df_rec.iloc[1]['Flowering period']+ "\033[0m")
else:
    print("The flowering period of the plant is", "\033[1m"+"unknown"+"\033[0m")


df_rec['Leaf color'] = df_rec['Leaf color'].str.replace('Blad','')
if not pd.isna(df_rec.iloc[1]['Leaf color']):
    print("The leaf color of the plant is", "\033[1m"+df_rec.iloc[1]['Leaf color']+ "\033[0m")
else:
    print("The leaf color of the plant is", "\033[1m"+"unknown"+"\033[0m")

df_rec['Flower color'] = df_rec['Flower color'].str.replace('Bloem','')
if not pd.isna(df_rec.iloc[1]['Flower color']):
    print("The flower color of the plant is", "\033[1m"+df_rec.iloc[1]['Flower color']+ "\033[0m")
else:
    print("The flower color of the plant is", "\033[1m"+"unknown"+"\033[0m")


if not pd.isna(df_rec.iloc[1]['Evergreen']):
    print("In the winter, the plant is", "\033[1m"+df_rec.iloc[1]['Evergreen']+ "\033[0m")
else:
    print("Whether the plant is evergreen or not is", "\033[1m"+"unknown"+"\033[0m")
    
if not pd.isna(df_rec.iloc[1]['Plant growth']):
    print("The maximum growth height is","\033[1m"+ df_rec.iloc[1]['Plant growth'].astype(str),"cm"+ "\033[0m")
else:
    print("The maximum growth height is", "\033[1m"+"unknown"+"\033[0m")

if not pd.isna(df_rec.iloc[1]['Maintenance']):
    print("The required maintenance level is", "\033[1m"+df_rec.iloc[1]['Maintenance']+ "\033[0m")
else:
    print("The required maintenance level is", "\033[1m"+"unknown"+"\033[0m")
    
    
if not pd.isna(df_rec.iloc[1]['Maintenance description']):
    print("To properly maintain the plant, you must do the following:", "\033[1m"+df_rec.iloc[1]['Maintenance description']+ "\033[0m")
else:
    print("What you need to do to maintain your plant is", "\033[1m"+"unknown"+"\033[0m")


print("This is a picture of the plant:")
Image(url = df_rec.iloc[1]['Photo'])

Second recommended plant:
The name of the plant is Duifkruid Scabiosa columbaria
The type of the plant is Borderplanten
The plant attracts GeenVogels, Bijen, Vlinders, GeenEgels and GeenEekhoorns
The flowering period of the plant is September Augustus Juli Juni
The leaf color of the plant is Groen Grijs
The flower color of the plant is Blauw Paars
In the winter, the plant is Wintergroen
The maximum growth height is 40.0 cm
The required maintenance level is Laag
To properly maintain the plant, you must do the following: De plant sterft elke winter af en loopt in het voorjaar weer uit. Net voor het uitlopen kan het oude loof afgeknipt of weggehaald worden.
This is a picture of the plant:


In [52]:
print('\033[1m'+'Third recommended plant:'+ '\033[0m')

print("The name of the plant is","\033[1m"+ df_rec.iloc[2]["Name"]+ "\033[0m")
print("The type of the plant is", "\033[1m"+df_rec.iloc[2]['Type']+ "\033[0m")
print("The plant attracts", "\033[1m"+df_rec.iloc[2]['Birds']+", "+df_rec.iloc[2]['Bees']+", "+df_rec.iloc[2]['Butterflies']+", "+df_rec.iloc[2]['Hedgehogs']+" and "+df_rec.iloc[2]['Squirrels']+ "\033[0m")

if not pd.isna(df_rec.iloc[2]['Flowering period']):
    print("The flowering period of the plant is", "\033[1m"+df_rec.iloc[2]['Flowering period']+ "\033[0m")
else:
    print("The flowering period of the plant is", "\033[1m"+"unknown"+"\033[0m")


df_rec['Leaf color'] = df_rec['Leaf color'].str.replace('Blad','')
if not pd.isna(df_rec.iloc[2]['Leaf color']):
    print("The leaf color of the plant is", "\033[1m"+df_rec.iloc[2]['Leaf color']+ "\033[0m")
else:
    print("The leaf color of the plant is", "\033[1m"+"unknown"+"\033[0m")

df_rec['Flower color'] = df_rec['Flower color'].str.replace('Bloem','')
if not pd.isna(df_rec.iloc[2]['Flower color']):
    print("The flower color of the plant is", "\033[1m"+df_rec.iloc[2]['Flower color']+ "\033[0m")
else:
    print("The flower color of the plant is", "\033[1m"+"unknown"+"\033[0m")


if not pd.isna(df_rec.iloc[2]['Evergreen']):
    print("In the winter, the plant is", "\033[1m"+df_rec.iloc[2]['Evergreen']+ "\033[0m")
else:
    print("Whether the plant is evergreen or not is", "\033[1m"+"unknown"+"\033[0m")
    
if not pd.isna(df_rec.iloc[2]['Plant growth']):
    print("The maximum growth height is","\033[1m"+ df_rec.iloc[2]['Plant growth'].astype(str),"cm"+ "\033[0m")
else:
    print("The maximum growth height is", "\033[1m"+"unknown"+"\033[0m")

if not pd.isna(df_rec.iloc[2]['Maintenance']):
    print("The required maintenance level is", "\033[1m"+df_rec.iloc[2]['Maintenance']+ "\033[0m")
else:
    print("The required maintenance level is", "\033[1m"+"unknown"+"\033[0m")
    
    
if not pd.isna(df_rec.iloc[2]['Maintenance description']):
    print("To properly maintain the plant, you must do the following:", "\033[1m"+df_rec.iloc[2]['Maintenance description']+ "\033[0m")
else:
    print("What you need to do to maintain your plant is", "\033[1m"+"unknown"+"\033[0m")


print("This is a picture of the plant:")
Image(url = df_rec.iloc[2]['Photo'])

Third recommended plant:
The name of the plant is Sleutelbloem Primula secundiflora
The type of the plant is Borderplanten
The plant attracts Vogels, Bijen, Vlinders, GeenEgels and GeenEekhoorns
The flowering period of the plant is Mei Juni
The leaf color of the plant is Groen
The flower color of the plant is Paars
In the winter, the plant is NietWintergroen
The maximum growth height is 60.0 cm
The required maintenance level is unknown
To properly maintain the plant, you must do the following: De plant heeft geen bijzondere zorg nodig.
This is a picture of the plant:


In [53]:
print('\033[1m'+'Fourth recommended plant:'+ '\033[0m')

print("The name of the plant is","\033[1m"+ df_rec.iloc[3]["Name"]+ "\033[0m")
print("The type of the plant is", "\033[1m"+df_rec.iloc[3]['Type']+ "\033[0m")
print("The plant attracts", "\033[1m"+df_rec.iloc[3]['Birds']+", "+df_rec.iloc[3]['Bees']+", "+df_rec.iloc[3]['Butterflies']+", "+df_rec.iloc[3]['Hedgehogs']+" and "+df_rec.iloc[3]['Squirrels']+ "\033[0m")

if not pd.isna(df_rec.iloc[3]['Flowering period']):
    print("The flowering period of the plant is", "\033[1m"+df_rec.iloc[3]['Flowering period']+ "\033[0m")
else:
    print("The flowering period of the plant is", "\033[1m"+"unknown"+"\033[0m")


df_rec['Leaf color'] = df_rec['Leaf color'].str.replace('Blad','')
if not pd.isna(df_rec.iloc[3]['Leaf color']):
    print("The leaf color of the plant is", "\033[1m"+df_rec.iloc[3]['Leaf color']+ "\033[0m")
else:
    print("The leaf color of the plant is", "\033[1m"+"unknown"+"\033[0m")

df_rec['Flower color'] = df_rec['Flower color'].str.replace('Bloem','')
if not pd.isna(df_rec.iloc[3]['Flower color']):
    print("The flower color of the plant is", "\033[1m"+df_rec.iloc[3]['Flower color']+ "\033[0m")
else:
    print("The flower color of the plant is", "\033[1m"+"unknown"+"\033[0m")


if not pd.isna(df_rec.iloc[3]['Evergreen']):
    print("In the winter, the plant is", "\033[1m"+df_rec.iloc[3]['Evergreen']+ "\033[0m")
else:
    print("Whether the plant is evergreen or not is", "\033[1m"+"unknown"+"\033[0m")
    
if not pd.isna(df_rec.iloc[3]['Plant growth']):
    print("The maximum growth height is","\033[1m"+ df_rec.iloc[3]['Plant growth'].astype(str),"cm"+ "\033[0m")
else:
    print("The maximum growth height is", "\033[1m"+"unknown"+"\033[0m")

if not pd.isna(df_rec.iloc[3]['Maintenance']):
    print("The required maintenance level is", "\033[1m"+df_rec.iloc[3]['Maintenance']+ "\033[0m")
else:
    print("The required maintenance level is", "\033[1m"+"unknown"+"\033[0m")
    
    
if not pd.isna(df_rec.iloc[3]['Maintenance description']):
    print("To properly maintain the plant, you must do the following:", "\033[1m"+df_rec.iloc[3]['Maintenance description']+ "\033[0m")
else:
    print("What you need to do to maintain your plant is", "\033[1m"+"unknown"+"\033[0m")


print("This is a picture of the plant:")
Image(url = df_rec.iloc[3]['Photo'])

Fourth recommended plant:
The name of the plant is Bergaster Aster amellus
The type of the plant is Borderplanten
The plant attracts GeenVogels, Bijen, Vlinders, GeenEgels and GeenEekhoorns
The flowering period of the plant is Oktober September Augustus
The leaf color of the plant is Groen
The flower color of the plant is Paars Blauw
In the winter, the plant is NietWintergroen
The maximum growth height is 60.0 cm
The required maintenance level is Laag
To properly maintain the plant, you must do the following: De plant sterft elke winter af en loopt in het voorjaar weer uit. Net voor het uitlopen kan het oude loof afgeknipt of weggehaald worden.
This is a picture of the plant:


In [54]:
print('\033[1m'+'Fifth recommended plant:'+ '\033[0m')

print("The name of the plant is","\033[1m"+ df_rec.iloc[4]["Name"]+ "\033[0m")
print("The type of the plant is", "\033[1m"+df_rec.iloc[4]['Type']+ "\033[0m")
print("The plant attracts", "\033[1m"+df_rec.iloc[4]['Birds']+", "+df_rec.iloc[4]['Bees']+", "+df_rec.iloc[4]['Butterflies']+", "+df_rec.iloc[4]['Hedgehogs']+" and "+df_rec.iloc[4]['Squirrels']+ "\033[0m")

if not pd.isna(df_rec.iloc[4]['Flowering period']):
    print("The flowering period of the plant is", "\033[1m"+df_rec.iloc[4]['Flowering period']+ "\033[0m")
else:
    print("The flowering period of the plant is", "\033[1m"+"unknown"+"\033[0m")


df_rec['Leaf color'] = df_rec['Leaf color'].str.replace('Blad','')
if not pd.isna(df_rec.iloc[4]['Leaf color']):
    print("The leaf color of the plant is", "\033[1m"+df_rec.iloc[4]['Leaf color']+ "\033[0m")
else:
    print("The leaf color of the plant is", "\033[1m"+"unknown"+"\033[0m")

df_rec['Flower color'] = df_rec['Flower color'].str.replace('Bloem','')
if not pd.isna(df_rec.iloc[4]['Flower color']):
    print("The flower color of the plant is", "\033[1m"+df_rec.iloc[4]['Flower color']+ "\033[0m")
else:
    print("The flower color of the plant is", "\033[1m"+"unknown"+"\033[0m")


if not pd.isna(df_rec.iloc[4]['Evergreen']):
    print("In the winter, the plant is", "\033[1m"+df_rec.iloc[4]['Evergreen']+ "\033[0m")
else:
    print("Whether the plant is evergreen or not is", "\033[1m"+"unknown"+"\033[0m")
    
if not pd.isna(df_rec.iloc[4]['Plant growth']):
    print("The maximum growth height is","\033[1m"+ df_rec.iloc[4]['Plant growth'].astype(str),"cm"+ "\033[0m")
else:
    print("The maximum growth height is", "\033[1m"+"unknown"+"\033[0m")

if not pd.isna(df_rec.iloc[4]['Maintenance']):
    print("The required maintenance level is", "\033[1m"+df_rec.iloc[4]['Maintenance']+ "\033[0m")
else:
    print("The required maintenance level is", "\033[1m"+"unknown"+"\033[0m")
    
    
if not pd.isna(df_rec.iloc[4]['Maintenance description']):
    print("To properly maintain the plant, you must do the following:", "\033[1m"+df_rec.iloc[4]['Maintenance description']+ "\033[0m")
else:
    print("What you need to do to maintain your plant is", "\033[1m"+"unknown"+"\033[0m")


print("This is a picture of the plant:")
Image(url = df_rec.iloc[4]['Photo'])

Fifth recommended plant:
The name of the plant is Sleutelbloem Primula pulverulenta
The type of the plant is Borderplanten
The plant attracts GeenVogels, Bijen, Vlinders, GeenEgels and GeenEekhoorns
The flowering period of the plant is Mei Juni
The leaf color of the plant is Groen
The flower color of the plant is Roze Paars
In the winter, the plant is Wintergroen
The maximum growth height is 60.0 cm
The required maintenance level is Geen
To properly maintain the plant, you must do the following: De plant heeft geen bijzondere zorg nodig.
This is a picture of the plant:


# References
VibhavSurve09 (2021) Recommendation System Cosine Similarity (Version 1.0) [Source code]. https://github.com/VibhavSurve09/recommendation-system-cosine-similarity